In [1]:
import nltk
import pandas as pd


In [12]:
df=pd.read_csv("/content/SMSSpamCollection",sep="\t",names=["Label","Message"])

In [13]:
df.head()

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Label    5572 non-null   object
 1   Message  5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [15]:
df.isnull().sum()

Label      0
Message    0
dtype: int64

In [16]:
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
wordnet = WordNetLemmatizer()
ps = PorterStemmer()

### Data Preprocessing

---



#### Regular Experssion and Stemming


In [28]:
corpus = []
for i in range(0, len(df)):
  review = re.sub('[^a-zA-Z]',' ', df['Message'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem (word) for word in review if not word in stopwords.words ('english')]
  review = ' '.join(review)
  corpus.append(review)

#### Tfidf Vectorizer

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [33]:
X = df['Message']

In [34]:
cv = TfidfVectorizer()
X=cv.fit_transform(corpus).toarray()

In [35]:
y=pd.get_dummies(df['Label'])
y=y.iloc[:,1].values

### Spliting Train and Test data

In [36]:
from sklearn.model_selection import train_test_split


In [38]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=0)


### Model Fitting

In [39]:
from sklearn.naive_bayes import MultinomialNB

In [40]:
spam_detect_model =MultinomialNB().fit(X_train,y_train)

### Model Evaluation

In [41]:
from sklearn.metrics import accuracy_score, classification_report

In [44]:
# Predict
y_pred = spam_detect_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("\nClassification Report:")
print(report)

Accuracy: 0.9659192825112107

Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       955
           1       0.99      0.77      0.87       160

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.92      1115
weighted avg       0.97      0.97      0.96      1115

